In [ ]:
import pandas as pd
import numpy as np
import copy
import gc

## 作業一: Q15 PLA的data set
#### https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_math/hw1_15_train.dat

In [ ]:
data_path = 'your path'
ori_pla_df = pd.read_excel(data_path)

# function: error_rate_estimate
### 輸入參數:
#### candidate_w: 特徵向量，就是訓練出來的那條線
#### estimate_df: 驗證資料的DataFrame

### 輸出結果:
#### err_num: 預測錯誤資料筆數
#### err_rate: 錯誤率

In [ ]:
def error_rate_estimate(candidate_w, estimate_df):
    err_num = 0
    err_rate = 0

    for x in estimate_df.itertuples():
        vec = np.array([1] + list(x[1:-1])).reshape(1,5)
        sign = np.dot(vec, candidate_w)
        
        ## pass if predition same as label, accumlate if different
        if (sign > 0 and x[-1] == 1) or (sign <= 0 and x[-1] == -1):
            pass
        else:
            err_num = err_num + 1
    err_rate = err_num/estimate_df.shape[0]
    return (err_num, err_rate)

## PLA

In [ ]:
sample_times = 2000
sample_max_iter_times = 5

#notice initial weights effects the final error rate
init_weights = np.zeros([5, 1])
#init_weights = np.random.rand(5).reshape(5, 1)

update_limit = 50
learning_rate = 0.5

pla_df = ori_pla_df
update_rs= []

In [ ]:
for seed in range(sample_times):
    random_df = pla_df.sample(n=pla_df.shape[0], random_state=seed).reset_index(drop=True)
    init_weights = np.zeros([5, 1])
    update_ct = 0
    
    for ran in range(sample_max_iter_times):
        for x in random_df.itertuples():
            vec = np.array([1] + list(x[1:-1])).reshape(1,5)
            sign = np.dot(vec, init_weights)

            if (sign > 0 and x[-1] == 1) or (sign <= 0 and x[-1] == -1):
                pass
            else:
                update_ct = update_ct + 1
                init_weights = init_weights + x[-1]*vec.T*learning_rate
                
        err_num, err_rate = error_rate_estimate(init_weights, random_df)
        if err_num == 0:
            print('資料集完全分隔')
            break
        
    print('sample {} total update {} times'.format(seed, update_ct))
    update_rs.append(update_ct)

In [ ]:
sum(update_rs)/len(update_rs)

# pocket 
### 請注意！pocket與基本的PLA只有一個地方有差別，pocket會將每次更新的weight存起來，將錯誤最少的weight作為最終結果

## 作業一: Q18 pocket的data set
#### https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_math/hw1_18_train.dat
#### https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_math/hw1_18_test.dat

In [ ]:
pocket_data_path = 'your path'
pocket_train_df = pd.read_excel(pocket_data_path, sheet_name='train')
pocket_test_df = pd.read_excel(pocket_data_path, sheet_name='test')

In [ ]:
#ff = 'your path'
#ori = pd.read_csv(ff,  delim_whitespace=True, names=['w1', 'w2', 'w3', 'w4', 'label'])
#ori = pd.read_csv(ff,  sep='\s+', names=['w1', 'w2', 'w3', 'w4', 'label'])

# random visit example with pocket

In [ ]:
sample_times = 2000
sample_max_iter_times = 5

#notice initial weights effects the final error rate
init_weights = np.zeros([5, 1])
#init_weights = np.random.rand(5).reshape(5, 1)

update_limit = 50
learning_rate = 0.5

pla_df = ori_pla_df
update_rs= []
err_rate = []

In [ ]:
for seed in range(sample_times):
    pla_df = pocket_train_df.copy()
    init_weights = np.zeros([5,1])
    #init_weights = np.random.rand(5).reshape(5, 1)
    
    min_weights = init_weights
    min_err, min_err_rate = error_rate_estimate(candidate_w=init_weights, estimate_df=pocket_train_df)
    
    update_ct = 0
    
    #maxium rows are double(1k) random data
    for row in range(pla_df.shape[0]*2):
        x = pla_df.sample(n=1).values[0]
        vec = np.array([1] + list(x[:-1])).reshape(1,5)
        sign = np.dot(vec, init_weights)

        if (sign > 0 and x[-1] == 1) or (sign <= 0 and x[-1] == -1):
            pass
        else:
            update_ct = update_ct + 1
            init_weights = init_weights + x[-1]*vec.T
            crr_err, crr_err_rate = error_rate_estimate(candidate_w=init_weights, estimate_df=pocket_train_df)
            
            #only keep minium error weight
            if min_err > crr_err:
                min_weights = init_weights
                
                #copy the integer varible value(pass by value) instead of pass by reference
                min_err = copy.copy(crr_err)
                min_err_rate = copy.copy(crr_err_rate)

            if update_ct == update_limit:
                #print('sample {} total update {} times'.format(seed, update_ct))
                break
    #notify w parameter's value, init or min           
    test_err, test_err_rate = error_rate_estimate(candidate_w=min_weights, estimate_df=pocket_test_df)
    
    print('{} times epoch, test error is {}, test error rate is {}.'.format(seed, test_err, test_err_rate))
    err_rate.append(test_err_rate)

In [ ]:
sum(err_rate)/len(err_rate)